In [ ]:
import os 
os.chdir("../")

In [ ]:
%pwd

In [1]:
import sys, platform
print("Python exe:", sys.executable)
print("Python ver:", platform.python_version())

Python exe: C:\Users\akuns\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe
Python ver: 3.13.9


In [2]:
import langchain
import pypdf

from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

print("✅ LangChain berhasil diimport!")
print("✅ PyPDF berhasil diimport!")

✅ LangChain berhasil diimport!
✅ PyPDF berhasil diimport!


In [4]:
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader

def load_pdf_files(data):
    loader = DirectoryLoader(
        data,
        glob="*.pdf",
        loader_cls=PyPDFLoader
    )
    documents = loader.load()
    return documents
    print("data berhasil di load")

In [5]:
extracted_data = load_pdf_files("data")

In [6]:
extracted_data

[Document(metadata={'producer': 'Microsoft® Word 2021', 'creator': 'Microsoft® Word 2021', 'creationdate': '2025-10-29T11:39:01+07:00', 'author': 'Amar Al Farizi', 'moddate': '2025-10-29T11:39:01+07:00', 'source': 'data\\data.pdf', 'total_pages': 3, 'page': 0, 'page_label': '1'}, page_content='Wisata Alam Banyumas \n1. Lokawisata Baturraden  terletak di Karangmangu, Baturraden . Lokawisata \nBaturraden merupakan destinasi utama di kaki Gunung Slamet yang menawarkan \ntaman rekreasi keluarga, kolam air panas, dan area bermain anak. Udara sejuk \npegunungan menjadikannya tempat ideal untuk berlibur bersama keluarga. Harga \ntiket dewasa R p20.000 (weekday), anak -anak Rp10.000 (weekday), dan weekend \nRp25.000 (dewasa), Rp12.500 (anak). Jam buka 7 AM dan jam tutup 7 PM.  \n2. Telaga Sunyi berlokasi di Ketenger, Baturraden. Telaga Sunyi adalah kolam alami \ndengan air jernih di tengah hutan pinus. Suasananya tenang dan sejuk, cocok untuk \nbersantai dan menikmati keindahan alam tanpa kera

In [7]:
len (extracted_data)

3

In [8]:
from langchain_core.documents import Document
from typing import List

def filter_to_minimal_docs(docs: List[Document]) -> List[Document]:
    """Simpan hanya page_content dan metadata 'source'."""
    minimal = []
    for d in docs:
        minimal.append(
            Document(
                page_content=d.page_content,
                metadata={"source": d.metadata.get("source")}
            )
        )
    return minimal

# === pakai fungsi & CEK hasilnya ===
min_docs = filter_to_minimal_docs(extracted_data)

# cek 1 dokumen
print(min_docs[0].metadata)          # -> {'source': 'data\\data.pdf'}
# pastikan tidak ada kunci lain:
print(list(min_docs[0].metadata.keys()))



{'source': 'data\\data.pdf'}
['source']


In [9]:
minimal_docs = filter_to_minimal_docs(extracted_data)

In [10]:
extracted_data

[Document(metadata={'producer': 'Microsoft® Word 2021', 'creator': 'Microsoft® Word 2021', 'creationdate': '2025-10-29T11:39:01+07:00', 'author': 'Amar Al Farizi', 'moddate': '2025-10-29T11:39:01+07:00', 'source': 'data\\data.pdf', 'total_pages': 3, 'page': 0, 'page_label': '1'}, page_content='Wisata Alam Banyumas \n1. Lokawisata Baturraden  terletak di Karangmangu, Baturraden . Lokawisata \nBaturraden merupakan destinasi utama di kaki Gunung Slamet yang menawarkan \ntaman rekreasi keluarga, kolam air panas, dan area bermain anak. Udara sejuk \npegunungan menjadikannya tempat ideal untuk berlibur bersama keluarga. Harga \ntiket dewasa R p20.000 (weekday), anak -anak Rp10.000 (weekday), dan weekend \nRp25.000 (dewasa), Rp12.500 (anak). Jam buka 7 AM dan jam tutup 7 PM.  \n2. Telaga Sunyi berlokasi di Ketenger, Baturraden. Telaga Sunyi adalah kolam alami \ndengan air jernih di tengah hutan pinus. Suasananya tenang dan sejuk, cocok untuk \nbersantai dan menikmati keindahan alam tanpa kera

In [11]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def text_split(minimal_docs):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=20,
    )
    texts_chunk = text_splitter.split_documents(minimal_docs)
    return texts_chunk

In [12]:
texts_chunk = text_split(minimal_docs)
print(f"Number of chunks: {len(texts_chunk)}")

Number of chunks: 20


In [13]:
texts_chunk

[Document(metadata={'source': 'data\\data.pdf'}, page_content='Wisata Alam Banyumas \n1. Lokawisata Baturraden  terletak di Karangmangu, Baturraden . Lokawisata \nBaturraden merupakan destinasi utama di kaki Gunung Slamet yang menawarkan \ntaman rekreasi keluarga, kolam air panas, dan area bermain anak. Udara sejuk \npegunungan menjadikannya tempat ideal untuk berlibur bersama keluarga. Harga \ntiket dewasa R p20.000 (weekday), anak -anak Rp10.000 (weekday), dan weekend \nRp25.000 (dewasa), Rp12.500 (anak). Jam buka 7 AM dan jam tutup 7 PM.'),
 Document(metadata={'source': 'data\\data.pdf'}, page_content='2. Telaga Sunyi berlokasi di Ketenger, Baturraden. Telaga Sunyi adalah kolam alami \ndengan air jernih di tengah hutan pinus. Suasananya tenang dan sejuk, cocok untuk \nbersantai dan menikmati keindahan alam tanpa keramaian. Harga tiket dewasa \n15000, anak-anak Tidak Ada, weekend Tidak Ada. Jam buka 8 AM dan jam tutup 4 PM. \n3. Hutan Pinus Limpakuwus berlokasi di Jl Baturraden Timur

In [14]:
from langchain.embeddings import HuggingFaceEmbeddings

def download_embeddings():
    """
    Download and return the HuggingFace embeddings model.
    """
    model_name = "sentence-transformers/all-MiniLM-L6-v2"
    embeddings = HuggingFaceEmbeddings(
        model_name=model_name,
        model_kwargs={"device": "cpu"}   # atau "cuda" jika pakai GPU
    )
    return embeddings

embeddings = download_embeddings()

C:\Users\akuns\AppData\Local\Temp\ipykernel_32236\694705367.py:8: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

C:\Users\akuns\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\akuns\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [15]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False, 'architecture': 'BertModel'})
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={'device': 'cpu'}, encode_kwargs={}, multi_process=False, show_progress=False)

In [24]:
daftar_teks = ["Hallo dunia"]
vectors = embeddings.embed_documents(daftar_teks)

print(vectors)

[[-0.03803442418575287, 0.0045180912129580975, -0.06060055270791054, -0.011571231298148632, -0.10467271506786346, -0.02041025273501873, 0.11283577978610992, -0.06815134733915329, -0.021047640591859818, 0.04313815385103226, -0.001139165135100484, -0.034102585166692734, -0.051712874323129654, -0.024494564160704613, -0.033192653208971024, 0.06849575787782669, 0.02272293157875538, 0.12076645344495773, -0.06955041736364365, -0.03025602176785469, 0.07576902210712433, -0.027407189831137657, -0.04367559403181076, 0.08945556730031967, -0.055883362889289856, 0.017747115343809128, 0.09197573363780975, 0.06066660210490227, -0.08802049607038498, -0.10222356766462326, -0.07262266427278519, 0.1297529935836792, 0.021427176892757416, -0.023860633373260498, -0.022033071145415306, 0.012226248160004616, -0.02029859833419323, -0.011958613060414791, 0.05186288803815842, -0.017058728262782097, 0.0024729985743761063, -0.057931218296289444, 0.046606194227933884, -0.026811791583895683, 0.025415459647774696, -0.

In [22]:
print(f"Vector length: {len(vectors[0])}")

Vector length: 384


In [25]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [27]:
from dotenv import load_dotenv
import os

load_dotenv()

PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

if not PINECONE_API_KEY or not OPENAI_API_KEY:
    print("Error: Pastikan API key sudah diatur di file .env")
else:
    print("API keys berhasil dimuat!")

API keys berhasil dimuat!


In [28]:
from pinecone import Pinecone 
pinecone_api_key = PINECONE_API_KEY

pc= Pinecone(api_key=pinecone_api_key)

In [29]:
pc

In [30]:
from pinecone import ServerlessSpec 

index_name = "compare-models-chatbot"

if not pc.has_index(index_name):
    pc.create_index(
        name = index_name,
        dimension=384,  
        metric= "cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )


index = pc.Index(index_name)

In [35]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=texts_chunk,
    embedding=embeddings,
    index_name=index_name
)

In [37]:
from langchain_pinecone import PineconeVectorStore
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)